In [1]:
import numpy as np
import math
from scipy.optimize import curve_fit
from copy import deepcopy as dcopy
import matplotlib.pyplot as plt

---------------------------------------GEM 1 : ENCODER AZIMUTH CORRECTION-------------------------------------------

In [2]:
def main():
    
    #Coleta dados de input
    jlist, filelist, jset, jjset, numframes = user_inputs() #ok
    _, Dir1, _, _, Dir4 = directories() #ok
    tag, file_list, file_name_list,file_name = input_files(filelist) #ok
    n_files = len(file_list)
    
    # Para cada arquivo da filelist, faz as correções necessárias nos dados de azimut e
    # escreve os arquivos de output
    
    for file_name_index in range(n_files): 
        jframe, time, azimut = input_datarr(file_name_list[file_name_index],Dir1)#ok
        
        dpfazi,slope,dslope,j = azi_correction(jframe,azimut,time,numframes,tag,file_name_index,file_name)
        
        # Calcula e escreve nos arquivos de output as estatísticas relacionadas a rotação
        write_stat_rot(dpfazi,slope,dslope,j,file_name,Dir4)
        

In [3]:
def user_inputs():
    
    
    jlist = int(input("Single file (1) or Multiple files (2)?: \n"))
    print("\n")
    
    if jlist == 1:
        datetag = int(input("Enter the date of the year [yy_MM_dd] : \n"))
        jfilenum = int(input("Enter the UT time of the day [hhmmssml] : \n"))
        jj = 1
    else:
        filelist = input("ENTER the filelist name: \n")
        print("\n")
        jj = 0

    jset = int(input(" SET Vmax? [1] => yes / [2] => no \n"))
    print("\n")

    if jset == 1:
        Vmax = float(input(" Write Vmax: \n"))
    else:
        Vmax = 9.8993 # 1999 observations

    jjset = int(input("SET slp rate? [1] => yes / [2] => no \n"))
    print("\n")
    if jjset == 1:
        slp = float(input("Write slp rate: \n"))
    else:
        slp = 17.0357 # iterated 1999 Vmax +/- 0.3918

    numframes = int(input("ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] \n"))
    
    return jlist, filelist, jset, jjset, numframes

In [4]:
def directories():
    
    " Diretórios."
    
    Dir = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_1_files/txt/" 
    Dir1 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_1_files/dat/"
    Dir2 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/rots/"
    Dir3 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/azimuth/"
    Dir4 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/rots/dpf/"
    
    return Dir, Dir1, Dir2, Dir3, Dir4

In [5]:
def input_files(filelist):
    
    " A partir das informações dadas pelo usuário, coleta os arquivos de input."

    Dir, Dir1, Dir2, Dir3, Dir4 = directories()
    with open(Dir + filelist) as arq: 
        filelist = arq.readlines()
        n_files = len(filelist)
        sub_file_list, datetag, tag, file_name_list = [], [], [], []
        for file_name_index in range(n_files):
            s_datetag = str(filelist[file_name_index])
            datetag.append(s_datetag[:8])
            tag.append(s_datetag)
            raw_name = filelist[file_name_index]
            sub_file_list.append(raw_name)
            #----------------------------
            file_name = raw_name
            file_name = file_name.replace("\t", "")
            file_name = file_name.replace("\n", "")
            file_name_list.append(file_name)
#             with open(Dir1 + file_name, "r+") as f: 
#                 input_data = f.readlines()
#                 n_lines = len(input_data)
        
    return tag, sub_file_list, file_name_list, file_name

In [6]:
def input_datarr(file_name,Dir1):
    
    " Coleta dados de input de cada um dos arquivos de uma determinada filelist."
 
    jframe, time, azimut = [],[],[]
    
    with open(Dir1 + file_name, "r+") as f: 
        input_data = f.readlines()
        n_lines = len(input_data)
        
    
    for line in input_data:
        input_line = line.replace(" ", "")
        input_line = line.replace("\n", "")
        input_line = input_line.split(" ")
        input_line = list(filter(None, input_line))
        jjframe = float(input_line[0])
        jtime = float(input_line[1])
        jazimut = float(input_line[4])
                
        jframe.append(jjframe)
        time.append(jtime)
        azimut.append(jazimut)
        
        #print("jframe",jframe)
        #print("time",time)
        #print("azimut",azimut)
        
    return jframe, time, azimut

In [7]:
def azi_correction(jframe,azimut,time,numframes,tag,file_name_index,file_name):
   
    " Para cada frame de um arquivo, verifica e corrige (com auxílio de outras funções) valores" 
    " de azimute incorretos."  
    
    print("Doing file:",tag[file_name_index])
    Dir,Dir1,Dir2,Dir3, Dir4 = directories()
    file_name = dcopy(tag[file_name_index])
    file_name = file_name.replace("\n","")
    
    timemean, aazmean, k, n_rotations, l, j105, j106, j = 0, 0, 0, 0, 0, 0, 0, 0
    
    aazimut, ttime, jccor, sig, dpfazi, speed, dspeed = [], [], [], [], [], [], []
    
    first, dangling, ic, new_rotation, first_time = True, False, None, False, True
    
    first_azi_new_rot = None
    
    i_frames = len(jframe)
    for i in range(i_frames): #for each line in file

        # Verficando até qual dado na lista de azimutes foi corrigido na rotação anterior.
        # A contagem de uma nova rotação deve iniciar no primeiro dado ainda não corrigido.
        # "first_azi_new_rot" é calculada no final de cada rotação, e indica onde a próxima
        # deve começar.
        
        if first_azi_new_rot is not None:
            if first_azi_new_rot != azimut[i]:
                first_azi_new_rot = None
                continue
                
        if first: # Get a reference in time and direction.
            compaz = azimut[i]   
            firstime = time[i]
            first = False
            
        #this is the expected behavior (azi-compaz<176), except for the very 
        #last azimut which may differ from the previous one
        #by < 176 and yet show reversal in sense of 
        #direction. The usual is a continuous decrease in 
        #azimut. An unexpected sign reversal can also happen. =>

        if (abs(azimut[i]-compaz)) < 176:
            l,ic,dangling,ttime,timemean,aazmean,sig,aazimut = azi_expected(i,105,aazimut,timemean,aazmean,\
            dangling,ttime,jccor,sig,time,azimut,compaz,l,ic,new_rotation)
            new_rotation = False
        else:
            num_fr_per_rot = len(aazimut)
            if num_fr_per_rot >= (numframes-1):
                j,slope,dslope,speed,dspeed,lastazi,midazi,firstazi,dpfazi_in_this_fr,l2,dpfazi,last_corr_azi,\
                first_azi_new_rot = azi_signal_reverse(numframes,dangling,aazimut,\
                timemean,aazmean,ttime,azimut,l,i,ic,j,\
                dpfazi,speed,dspeed,j105,j106,first_time,jccor) 
                
                first_time = False # Indica quando a função "azi_signal_reverse" deve iniciar a contagem
                # de dados que passam por ela. * Reinicia a cada rotação.
                
                # write in rot file (descrições dos arq. a escrever aq)
                dat_list_rot = [j,slope,dslope,lastazi,midazi,firstazi,dpfazi_in_this_fr]
                rot_type = "rot.dat"
                mode_rot = True
                aazi_dat = False
                write_output(dat_list_rot,file_name,Dir2,rot_type,mode_rot,aazi_dat)
                
            else:
                slope,dslope,aazi,l2,last_corr_azi,first_azi_new_rot = azi_incomplete_rotations(timemean,aazmean,ttime,aazimut,azimut,l,ic,i)
                l_incomplete_rot = dcopy(l)
                aazi_last_incomplete_rot = dcopy(aazimut)
                print("l_incomplete_rot",l_incomplete_rot)
                
            aazimut,fpd = frame_p_degree(slope,l,l2,aazimut,ttime) 


            # write in azimut file
            dat_list_aazi = aazimut
#             print("l",l)
#             print("aazimut antes de write_output",aazimut)
#             print("\n\n")
            if (n_rotations) == 0: mode_aazi = True
            else: mode_aazi = False
            aazi_type = "azi.dat"
            aazi_dat = True
            print("file_name",file_name)
            write_output(dat_list_aazi,file_name,Dir3,aazi_type,mode_aazi,aazi_dat)
            
            # reset p/ próxima rotação 
            l,timemean,aazmean = 0,0,0 
            ic,dangling,new_rotation = None,False,True
            aazimut,ttime = [],[]
            n_rotations =+ 1
                
        if (abs(azimut[i]-compaz)) < 176:
            compaz = azimut[i] # atualiza o valor referência para o próx. azi dentro de uma rotação
        else:
            compaz = last_corr_azi  #atualiza o valor referência para a próxima rotação  
    
    # last incomplete rotation
    aazi_last_incomplete_rot = last_incpl_rotation(l_incomplete_rot,aazi_last_incomplete_rot,fpd,ttime,time)
    # write last incomplete rotation
    dat_list_azi = dcopy(aazi_last_incomplete_rot)
    if (n_rotations) == 0: mode_azi = True
    else: mode_azi = False
    azi_type = "azi.dat"
    aazi_dat = True
    write_output(dat_list_azi,file_name,Dir3,azi_type,mode_azi,aazi_dat)
    
    return dpfazi,speed,dspeed,j 

In [8]:
def azi_expected(i,numframes,aazimut,timemean,aazmean,dangling,ttime,jccor,sig,time,azimut,compaz,l,ic,new_rotation):
    
    " Verifica se há uma reversão no sinal no último azimut (da rotação),"
    " ou se há uma inversão inesperada "
#     print(len(ttime), len(aazimut), "azi_expected")
    
    if i == 0 or new_rotation:
        l = 0
    else:
        l += 1
    
    if (l > 0) and (time[i] - ttime[l-1] < 0):
        ttime.append(time[i] + 86400)
    else:
        ttime.append(time[i])
        
    aazimut.append(azimut[i])
    timemean = timemean + ttime[l]
    aazmean = aazmean + azimut[i]

    sig.append(1)
    
    len_aazi = len(aazimut)
    
    if (azimut[i]-compaz) > 0: #
        if l >= (numframes-1):
            dangling = True #sign reversal in last azimut detected 
        else:
            if ic == None: 
                ic = 0 #unexpected sign
            else: 
                ic += 1
            jccor.append(l)

    return l,ic,dangling,ttime,timemean,aazmean,sig,aazimut

In [9]:
def azi_signal_reverse(numframes,dangling,aazimut,timemean,aazmean,ttime,\
                       azimut,l,i,ic,j,dpfazi,speed,dspeed,j105,j106,first_time,jccor):
    " Caso o valor atual de azi tenha decrescido fora do esperado (azi-compaz > 176),"
    " mas o número atual de 'l' contabilizados corresponde a uma rotação completa,"
    " então será feita uma correção nos valores de azi em que ocorreram reversões inesperadas,"
    " a partir do valor calculado de slope (vel./s) na função dpf."
    
    lastazi, midazi, firstazi= 0, 0, 0
    frame_rot_counter = len(aazimut)
                                              #In case the sign reversal had already occured
    if frame_rot_counter == (numframes-1):    #in the previous frame but the magnitude fell 
        j105 += 1                             #short of 176, we delete it from the mean (mm=1)
    if frame_rot_counter == numframes:        #double-dangling forces l to increase by 1. Even
        j106 += 1                             #when lastazi is really only the l-1 element, mid
                                              #azi serves just as well as firstazi to set dpfazi
    
    if first_time == True:
        j = 0  
    else: 
        j+=1 
        
    if dangling:# Se ocorreu reversão no último frame
        mm = 1
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm,i)
        lastazi = aazimut[l-2] 
        midazi = aazimut[l-1]
        firstazi = aazimut[l]
        l2 = 1
#         print("l",l)
#         print("lastazi","midazi","firstazi",lastazi,midazi,firstazi)

    else: # Se não
        lastazi = aazimut[l]
        mm = 0
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm,i)
        midazi = aazimut[l]
        firstazi = aazimut[l+1]
        l2 = 2
        
    speed.append(slope)
    dspeed.append(dslope)

    #If the Vmax used is correct, then midazi will be
    #spaced evenly from lastazi to firstazi according
    #to the prevailing slope. So we check and correct
    
    dpfazi_value = (360.0 + lastazi - firstazi)/2 # Calculando vel. graus/frame
    dpfazi.append(dpfazi_value)

    dpfazi_in_this_fr = (360.0 + lastazi - firstazi)/2
    
    # ic value = times unexpected signal reverse occurs (first time => ic = 0) 
    if not ic == None : 
    # Substituindo valores de azimute onde aconteceram reversões inesperadas, pelo valor corrigido 
    # de (vel./s * tempo_entre_frames = azi_corrigido)
        count = 0
        while count <= ic:
            aazimut[(jccor[count])] = aazimut[(jccor[count] - 1)] + (slope*0.56002)
            if aazimut[(jccor[count])] < 0: aazimut[(jccor[count])] += 360.0
            count += 1
    print("aazimut azi sig rev",aazimut)
    print("jccor",jccor)
    print("count",count)
    print("\n\n")

    return j,slope,dslope,speed,dspeed,lastazi,midazi,firstazi,dpfazi_in_this_fr,l2,dpfazi,last_corr_azi,first_azi_new_rot # write in rots file

In [10]:
def azi_incomplete_rotations(timemean,aazmean,ttime,aazimut,azimut,l,ic,i):
    
    " Caso 'l' corresponda a uma rotação incompleta, o valor de slope também será calculado. "
    
    print("Incomplete rotation of l frames =>",l)
    
    #print("timemean - azi_inc_rot_func",timemean)
    
    if not ic == None:
        mm = 1
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm,i)
#         print("azimut[i]",azimut[i])
#         print("slope azi_inc_rot",slope)
        l2 = 1
    else:
        mm = 0
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm,i)
        l2 = 2
    
    return slope,dslope,aazi,l2,last_corr_azi,first_azi_new_rot

In [11]:
def frame_p_degree(slope,l,l2,aazimut,ttime):
    
    " Calcula fpd (degree/frame), a partir do valor de slope, e realiza uma correção nos "
    " valores de azi que não forem compatíveis, numa margem, com o azimute calculado a partir de (vel*tempo)."
    
    fpd = slope*0.56002 
    
    print("fpd",fpd)
#     print("len(ttime)",len(ttime))
#     print("l+l2",l+l2) 
#     print("l2",l2)  
#     print("ttime",ttime)

    d = 0
    while d <= (l+l2):
        if d >= (len(ttime)):
            ttime.append(0)
        if d > 0:
            aa_zi = abs(aazimut[d] - aazimut[d-1]) 
            tti_me = (ttime[d] - ttime[d-1])
            
            if (aa_zi < (abs(fpd/2))) or (aa_zi > (3*(abs(fpd/2)))) and (tti_me < (3*0.56002/2)):
                
                aazimut[d] = aazimut[d-1] + fpd 
        d += 1
        
    return aazimut,fpd

In [12]:
def last_incpl_rotation(l,aazimut,fpd,ttime,time):
    
    " Correção dos valores da última rotação."
    print("l last_incpl_rot",l)
    i = 0
    while i <= l: #last incomplete rotation 
        if i > 0:
            
            if ((aazimut[i] - aazimut[i-1]) < (fpd/2)) or ((aazimut[i] - aazimut[i-1]) > (3*fpd/2))\
            and ((ttime[i] - time[i-1]) < (3*0.56002/2)):
                aazimut[i] = aazimut[i-1] + fpd
        i += 1
        
    return aazimut

In [13]:
def write_output(listt, file_name, dirr, type_file, mode, aazi_dat):
    """
    Escreve os dados processados em seus respectivos arquivos.
    """
    if aazi_dat:
        new_name = file_name.replace(".dat", type_file)
        write_mode = "w" if mode else "a"  # Define o modo de escrita (sobrescrever ou anexar)

        with open(dirr + new_name, write_mode) as arq_output:
            for item in listt:
                arq_output.write(f"{item:.6f}\n")  # Escreve cada item em uma linha, formatado com 2 casas decimais

    else:
        
        data = str(listt[0])
    #     print("listt",listt)
        for item in listt:
            item = f"{item:.2f}"
            data += f"\t{item}\t"
        new_name = file_name.replace(".dat", type_file)

        if mode:
            with open(dirr + new_name,"w") as arq_output:
                arq_output.write(data)
                arq_output.write("\n")
        else:
            print("dirr+new_name",dirr+new_name)
            with open(dirr + new_name,"a") as arq_output:
                arq_output.write(data)
                arq_output.write("\n")

In [14]:
def write_stat_rot(dpfazi,speed,dspeed,j,file_name,Dir4):
    
    slopmean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,\
    mean_per,per_sigman,sigma_per,sigint_per,Vslp = stats_parameters(dpfazi,speed,dspeed,j)
    
    mode_rot_dpf = True
    listt_dpf = [fpd,sig_dpf,dpfmean,sigdpf,(fpd-dpfmean),Vslp]
    type_dpf = "dpf_rots.dat"
    dirr_dpf = Dir4
    aazi_dat = False
    write_output(listt_dpf,file_name,Dir4,type_dpf,mode_rot_dpf,aazi_dat)

In [15]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::				
def stats_parameters(dpfazi,speed,dspeed,j):
    """
    Calculates rotation statistics
    """
    Vmax, slp = 9.8993,17.0357 # for 1999 observations
    wt, slopemean, dpfmean, sigma, sigdpf = 0,0,0,0,0
    for i in range(j):
        w = 1.0/dspeed[i]**2
        slopemean += speed[i]*w
        wt += w 
        dpfmean += dpfazi[i]
    #print("w",w)
    slopemean = slopemean/wt #mean dish rotation speed in deg/sec
    dpfmean = dpfmean/j ##Average dpf speed in deg/frame
    for i in range(j):
        w = 1.0/dspeed[i]**2
        sigma = sigma + w*(speed[i] - slopemean)**2    
        sigdpf = sigdpf + (dpfazi[i] - dpfmean)**2

    sigma = math.sqrt(sigma/wt*j/(j-1)) # sigma dispersion (slopemean) deg/sec
    sigman = sigma/math.sqrt(j*1) # external error (slopemean) deg/sec
    sigint = math.sqrt(1/wt) # internal error (slopemean) deg/sec            
    Z = sigman/sigint # z statistic
    dZ = 1/math.sqrt(2*(j-1))         
    sigdpf = math.sqrt(sigdpf/(j-1))/math.sqrt(j*1)

    fpd = slopemean*0.56002 # frames/degree
    sig_dpf = sigma * 0.56002 # sigma dispersion
    sigman_dpf = sigman * 0.56002 # external error
    sigint_dpf = sigint * 0.56002 # internal error

    mean_per = 360/slopemean # Average rotation period in seconds
    per_sigman = 360*sigman/slopemean**2 # external error
    sigma_per = 360*sigma/slopemean**2 # sigma dispersion
    sigint_per = 360*sigint/slopemean**2 # internal error

    Vslp = Vmax + (fpd - dpfmean)/slp
    return(slopemean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,mean_per,per_sigman,sigma_per,sigint_per,Vslp)

In [16]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def dpf(tmean,zmean,ttim,aazi,azimut,n,m,i):
    """
    Using the output of the least_sq function and the average values 
    of time and azimuth, calculate the speed in degrees/frame
    """

    if m == 0: #se não há inversão de sinal 
                            ##  tmean é inicialmente a soma de todos os valores de tempo 
        tmean = tmean/(n+1) ##  logo, aqui temos a média do tempo de uma rotação
        zmean = zmean/(n+1) ##  mesma coisa para o azimute
       
    else: #se há
        tmean = (tmean - ttim[n])/n
        zmean = (zmean - aazi[n])/n
    
    print("i dpf",i)

    i = 0 ##
    while i <= n-m: ## subtrai valor médio em todos os dados, menos o último (se houver inversão)
        ttim[i] = (ttim[i] - tmean)
        aazi[i] = (aazi[i] - zmean)
        i += 1
    
    if ((n+1)-m) > 2:
        lsq_azi, lsq_tim = dcopy(aazi),dcopy(ttim)
        lsq_azi[-1],lsq_tim[-1] = 0, 0
        slope, zer, dslope, dzer  = least_sq(function, lsq_tim, lsq_azi) # ajuste linear(tempo,azimute)
    else:
        slope, dslope = 0, 0
    
    azi = dcopy(azimut[i])
    aazi.append(azi)
    
    print("slope",slope) # vel angular
    print("dslope",dslope) # incerteza
    
    i = 0
    while i <= n-m:
        aazi[i] += zmean
        i += 1
    
    # Código de ajuste dos valores de azimute corrigidos
#     if m == 0:
#         jk = ((n + 2) - m)- 1
#         next_azi = azimut[i+2]
#         first_azi_new_rot = azimut[i+3]
#     else:
#         jk = (n + 2) - m
   
#     print("len(aazi)",len(aazi))
    jk = (n + 2) - m
    if jk >= len(aazi):
        aazi.append(0)
    
    next_azi = dcopy(azimut[i+1])
    print("next_azi",next_azi)
    print("aazi array",aazi)
    print("azi", azi)
    print("\n\n")
    print("m",m)
    print("n",n)

    if (aazi[jk] <= next_azi) or (abs(aazi[jk] - next_azi) < (abs(slope * 0.56002 / 8 * 7))):
        dstep = (360 + aazi[n - m] - next_azi) / 3
        for i in range(1, 4):
            if i - m > 0:
                aazi.append(0)  # aazi é estendido para armazenar os valores
            aazi[n + i - m] = aazi[n - m] - i * dstep
            if aazi[n + i - m] < 0:
                aazi[n + i - m] += 360
    else:
#         print("verf dpf aazi[n+1-m]",aazi[n + 1 - m])
#         print("verf dpf aazi[n+2-m]",aazi[n + 2 - m])
        aazi[n + 1 - m] = aazi[n - m] - (360 + aazi[n - m] - aazi[n + 2 - m]) / 2
        if aazi[n + 1 - m] < 0:
            aazi[n + 1 - m] += 360

        if (n + 3 - m) >= len(aazi):
            aazi.append(0)  

        # Atualiza o valor corrigido
        aazi[n + 3 - m] = next_azi
#         print("verf dpf aazi[n+3-m]",aazi[n + 3 - m])
    if aazi and aazi[-1] == 0:
        aazi.pop()
        
    last_corr_azi = dcopy(next_azi)
    first_azi_new_rot = dcopy(azimut[i+2])
#     print("aazi array após correção",aazi)
#     print("\n\n")
    n += 1

    return slope,dslope,aazi,first_azi_new_rot,last_corr_azi

In [17]:
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::	
def least_sq(function,x,y):
    """
    Calculate a linear fit to the azimuth data as a function of time,
    obtaining the speed in degrees/sec and their respective uncertainties
    """
    x = np.array(x, dtype="f8")
    y = np.array(y, dtype="f8")
    param, m_cov = curve_fit(function, x, y) 
    slope, intercept = param

    perr = np.sqrt(np.diag(m_cov)) #siga, sigb
    
#     plt.scatter(x, y, color='blue', label='Dados')  
#     plt.plot(x, function(x, slope, intercept), color='red',label=f'Ajuste: y = {slope:.2f}x + {intercept:.2f}')  
#     plt.xlabel('Eixo X')
#     plt.ylabel('Eixo Y')
#     plt.legend()
#     plt.grid(True)
#     plt.show()
    
    return param[0],param[1],perr[0],perr[1]
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [18]:
def function(x,a,b):
    return a*x + b  

#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [19]:
main()

Single file (1) or Multiple files (2)?: 
2


ENTER the filelist name: 
fileli001.txt


 SET Vmax? [1] => yes / [2] => no 
2


SET slp rate? [1] => yes / [2] => no 
2


ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] 
105
Doing file: 99_05_1820234490.dat

Incomplete rotation of l frames => 59
i dpf 60
slope -6.020441698489771
dslope 0.0004336907186615813
next_azi 343.18
aazi array [198.086, 194.691, 191.373, 187.956, 184.66, 181.276, 177.87, 174.486, 171.146, 167.773, 164.433, 160.983, 157.699, 154.248, 150.953, 147.58, 144.162, 140.834, 137.461, 134.099, 130.759, 127.319, 124.002, 120.607, 117.245, 113.883, 110.466, 107.093, 103.708, 100.358, 97.007, 93.556, 90.261, 86.855, 83.493, 80.12, 76.702, 73.407, 70.034, 66.661, 63.288, 59.893, 56.553, 53.147, 49.785, 46.368, 42.995, 39.633, 36.248999999999995, 32.876000000000005, 29.503, 26.14099999999999, 22.801000000000002, 19.406000000000006, 16.099000000000004, 12.659999999999997, 9.3419999999999

fpd -3.371810629288628
file_name 99_05_1820234490.dat
i dpf 7537
slope -6.02112984441023
dslope 0.00022492080397090836
next_azi 201.07
aazi array [351.024, 347.696, 344.378, 340.961, 337.643, 334.226, 330.886, 327.513, 324.118, 320.767, 317.417, 313.988, 310.659, 307.286, 303.925, 300.552, 297.123, 293.817, 290.388, 287.06, 283.675, 280.236, 276.907, 273.545, 270.139, 266.811, 263.426, 260.053, 256.68, 253.285, 249.979, 246.539, 243.2, 239.849, 236.476, 233.047, 229.696, 226.301, 222.95, 219.577, 216.182, 212.854, 209.447, 206.108, 202.735, 199.373, 196.011, 192.616, 189.309, 185.903, 182.541, 179.235, 175.795, 172.422, 169.049, 165.665, 162.348, 158.908, 155.579, 152.162, 148.844, 145.494, 142.032, 138.748, 135.375, 131.991, 128.607, 125.245, 121.894, 118.465, 115.126, 111.742, 108.346, 105.04, 101.556, 98.25, 94.832, 91.492, 88.108, 84.691, 81.373, 77.989, 74.65, 71.255, 67.87, 64.52, 61.09100000000001, 57.774, 54.355999999999995, 50.961, 47.644000000000005, 44.20400000000001, 40.831

fpd -3.3720919556500295
file_name 99_05_1820234490.dat
i dpf 13946
slope -6.020422052040267
dslope 0.00019773492727083968
next_azi 204.366
aazi array [349.549, 346.253, 342.858, 339.463, 336.101, 332.706, 329.4, 326.015, 322.598, 319.281, 315.852, 312.512, 309.173, 305.777, 302.427, 298.998, 295.647, 292.285, 288.857, 285.562, 282.111, 278.76, 275.376, 271.981, 268.652, 265.246, 261.929, 258.544, 255.149, 251.798, 248.437, 245.086, 241.735, 238.329, 234.989, 231.583, 228.232, 224.826, 221.464, 218.08, 214.707, 211.356, 207.994, 204.643, 201.248, 197.908, 194.491, 191.173, 187.778, 184.427, 181.088, 177.648, 174.331, 170.946, 167.551, 164.223, 160.816, 157.466, 154.07, 150.664, 147.347, 143.963, 140.59, 137.25, 133.832, 130.493, 127.12, 123.736, 120.396, 116.979, 113.65, 110.233, 106.848, 103.509, 100.125, 96.741, 93.356, 89.984, 86.622, 83.26, 79.876, 76.536, 73.141, 69.779, 66.417, 63.010999999999996, 59.682, 56.27600000000001, 52.891999999999996, 49.53, 46.146000000000015, 42.762, 39

IndexError: list index out of range